In [ ]:
import os
import subprocess
from fileoperations.fileoperations import getFileNamesInDir
import pdb

In [ ]:
folder = '../txt'
keyword = '*.txt'
files, dummy, names = getFileNamesInDir(folder, keyword=keyword)
names = [os.path.splitext(n)[0] for n in names]


In [ ]:
iconv_map = {'utf-16le':'UTF-16', 'Little-endian UTF-16 Unicode':'UTF-16',
             'iso-8859-1':'ISO_8859-9','ISO-8859':'ISO_8859-9'}
for f,n in zip(files, names):
    try:  # unix
        out = subprocess.check_output("file -i " + f, shell = True)
        curr_charset = out.split('charset=')[1]

        if curr_charset.endswith('\n'):
            curr_charset = curr_charset[:-1]
            
        if not any(curr_charset in charset for charset in ['utf-8', 'us-ascii']):
            print curr_charset + '\t' + n
            commandstr = ("iconv -f " +  iconv_map[curr_charset] + " -t UTF-8 " + f + " > tmp.txt "
                          "&& mv -f tmp.txt " + f)
            out = subprocess.check_output(commandstr, shell = True)
            
        # also change the line break from \r\n to \n
        out = subprocess.check_output("fromdos " + f, shell = True)
    except IndexError:  # mac
        out = subprocess.check_output("file -b " + f, shell = True)
        curr_charset = out.split(' text')[0]

        if not any(curr_charset in charset for charset in ['UTF-8 Unicode', 'ASCII']):
            print curr_charset + '\t' + n
            commandstr = ("iconv -f " +  iconv_map[curr_charset] + " -t UTF-8 " + f + " > tmp.txt "
                          "&& mv -f tmp.txt " + f)
            out = subprocess.check_output(commandstr, shell = True)
        
        # also change the line break from \r\n to \n
        out = subprocess.check_output("sed -e 's/\r$//' " + f + " > tmp.txt "
                                      "&& mv -f tmp.txt " + f, shell = True)
